# Libraries installation on the current python environment

# Pre-processing original data

In [103]:
import csv
def custom_header_reader(file:'TextIOWrapper'):
    csvReader = csv.reader(file,delimiter='\t')
    for row, text in enumerate(csvReader):
        if row == 2: numMarkers = int(text[-1])
        elif row == 10: 
            columnNames = text[:-1]
            if columnNames[0] == 'Frame': 
                break
        elif row == 11: columnNames = text[:-1]; break
    return numMarkers, columnNames, csvReader

def line_reader(csvReader,fromSecond,toSecond):
    for line in csvReader:
        if fromSecond <= float(line[1]) <= toSecond:
            yield line[:len(columnNames)]
        elif float(line[1]) > toSecond:
            break

In [104]:
sampleAnno = 3

## Reading from dataset

In [105]:
import os
import pandas as pd
ANNOTATIONS_PATH = 'data'
RAW_PATH = ANNOTATIONS_PATH+'/raw'
#with open(ANNOTATIONS_PATH+'/annotations.txt','r') as file:
#    annotations = file.read().splitlines()
annotations_file = ANNOTATIONS_PATH+'/annotationsVExamples.txt' #'/annotationsVSingle.txt'
#annotations_file = ANNOTATIONS_PATH+'/annotationsVSingle.txt'
with open(annotations_file,'r') as file:
    annotations = file.read().splitlines()[1:]#[line for line in file.read().splitlines() if 'no val' in line]
    print(len(annotations))

if '(A)' in annotations[sampleAnno] or '(B)' in annotations[sampleAnno]:
    folder, trial, fragId, side, OoM, startSec, endSec = annotations[sampleAnno].replace(' ','').split(',')[:7]
else:
    folder, trial, fragId, OoM, startSec, endSec = annotations[sampleAnno].replace(' ','').split(',')[:6]
sampleData = os.path.join(RAW_PATH,folder,trial+'.tsv')
print(sampleData,startSec,endSec)
with open(sampleData,'r') as file:
    numMarkers, columnNames, readerBuffer = custom_header_reader(file)
    data = pd.DataFrame(line_reader(readerBuffer,float(startSec),float(endSec)),columns=columnNames).astype(dict(zip(columnNames,[int,float,str]+[float]*len(columnNames[3:]))))
if data['Time'].iloc[0] > float(startSec) or data['Time'].iloc[-1] < float(endSec):
    raise Exception(f'\x1b[31mNOT FULL FRAMES!!! data starts at: {data["Time"].iloc[0]} of {float(startSec)} and ends at {data["Time"].iloc[-1]} of {float(endSec)}\x1b[0m')
else:
    print("\x1b[32mSTART AND END SECONDS CHECK CORRECT!\x1b[0m")
data.head()

4
data/raw/2020-07-08/t_052.tsv 8.29 15.49
START AND END SECONDS CHECK CORRECT!


,Frame,Time,SMPTE,RKneeIn_A X,RKneeIn_A Y,RKneeIn_A Z,RKneeIn_B X,RKneeIn_B Y,RKneeIn_B Z,RKneeOut_A X,...,RElbowOut_A Z,RElbowOut_B X,RElbowOut_B Y,RElbowOut_B Z,LKneeOut_A X,LKneeOut_A Y,LKneeOut_A Z,LKneeOut_B X,LKneeOut_B Y,LKneeOut_B Z
0,830,8.29,00:54:32:07,-1673.651,963.197,407.819,2048.209,861.935,402.985,-1730.089,...,991.214,2062.924,1096.356,921.177,-1652.642,1137.316,438.813,2042.820,711.856,412.678
1,831,8.30,00:54:32:08,-1673.886,962.429,408.036,2049.215,862.051,403.387,-1730.359,...,991.115,2064.530,1096.553,924.334,-1650.491,1136.101,438.681,2043.927,712.066,413.110
2,832,8.31,00:54:32:08,-1673.730,961.543,408.647,2050.223,862.154,403.958,-1730.601,...,990.909,2065.999,1096.807,927.365,-1648.053,1134.955,438.657,2045.124,712.410,413.440
3,833,8.32,00:54:32:08,-1673.931,960.643,408.877,2051.358,862.271,404.220,-1730.957,...,990.796,2067.422,1097.061,930.243,-1645.604,1133.850,438.622,2046.924,712.421,413.485
4,834,8.33,00:54:32:08,-1674.148,959.777,409.036,2053.068,862.517,404.661,-1731.181,...,990.582,2068.623,1097.272,933.191,-1642.804,1132.815,438.590,2048.922,712.305,413.393


## Standardizing column names

In [106]:
print(len(data))
data.iloc[:,3::3].head()

721


,RKneeIn_A X,RKneeIn_B X,RKneeOut_A X,RKneeOut_B X,LKneeIn_B X,HeadFront_B X,HeadFront_A X,LKneeIn_A X,HeadL_A X,HeadL_B X,...,LElbowIn_A X,LElbowIn_B X,RElbowIn_A X,RElbowIn_B X,LElbowOut_A X,LElbowOut_B X,RElbowOut_A X,RElbowOut_B X,LKneeOut_A X,LKneeOut_B X
0,-1673.651,2048.209,-1730.089,2044.336,2044.547,1805.992,-1579.140,-1667.155,-1669.114,1921.495,...,-1677.698,2054.654,-1685.379,2029.206,-1730.830,2038.960,-1739.300,2062.924,-1652.642,2042.820
1,-1673.886,2049.215,-1730.359,2045.427,2045.182,1811.099,-1579.041,-1664.778,-1669.038,1926.338,...,-1677.446,2056.426,-1685.422,2030.685,-1730.615,2041.259,-1739.384,2064.530,-1650.491,2043.927
2,-1673.730,2050.223,-1730.601,2048.730,2045.897,1816.164,-1578.918,-1661.537,-1669.021,1931.114,...,-1677.397,2058.120,-1685.558,2032.051,-1730.367,2043.429,-1739.348,2065.999,-1648.053,2045.124
3,-1673.931,2051.358,-1730.957,2049.588,2046.303,1821.097,-1578.777,-1658.845,-1668.993,1935.822,...,-1677.169,2059.575,-1685.663,2032.964,-1730.131,2044.951,-1739.403,2067.422,-1645.604,2046.924
4,-1674.148,2053.068,-1731.181,2050.326,2047.075,1825.894,-1578.692,-1655.908,-1668.920,1940.430,...,-1677.000,2060.725,-1685.778,2034.026,-1729.996,2046.656,-1739.449,2068.623,-1642.804,2048.922


In [107]:
[colname[:-2] for colname in list(data.iloc[:1,3::3].columns)]

['RKneeIn_A',
 'RKneeIn_B',
 'RKneeOut_A',
 'RKneeOut_B',
 'LKneeIn_B',
 'HeadFront_B',
 'HeadFront_A',
 'LKneeIn_A',
 'HeadL_A',
 'HeadL_B',
 'HeadR_A',
 'HeadR_B',
 'Chest_A',
 'Chest_B',
 'WaistLFront_A',
 'WaistLFront_B',
 'WaistRFront_A',
 'WaistRFront_B',
 'LShoulderTop_A',
 'LShoulderTop_B',
 'RShoulderTop_A',
 'RShoulderTop_B',
 'SpineThoracic2_A',
 'SpineThoracic2_B',
 'SpineThoracic12_A',
 'SpineThoracic12_B',
 'WaistL_A',
 'WaistL_B',
 'WaistR_A',
 'WaistR_B',
 'WaistBack_A',
 'WaistBack_B',
 'LHeelBack_A',
 'LHeelBack_B',
 'RHeelBack_A',
 'RHeelBack_B',
 'LAnkleOut_A',
 'LAnkleOut_B',
 'RAnkleOut_A',
 'RAnkleOut_B',
 'LForefoot2_A',
 'RForefoot2_A',
 'LForefoot2_B',
 'RForefoot2_B',
 'LForefoot5_A',
 'RForefoot5_A',
 'LForefoot5_B',
 'RForefoot5_B',
 'LHand2_A',
 'LHand2_B',
 'RHand2_A',
 'RHand2_B',
 'LWristIn_A',
 'LWristIn_B',
 'RWristIn_A',
 'RWristIn_B',
 'LWristOut_A',
 'RWristOut_A',
 'LWristOut_B',
 'RWristOut_B',
 'LElbowIn_A',
 'LElbowIn_B',
 'RElbowIn_A',
 'RElbo

In [108]:
import pandas as pd
if '_A' in data.columns[3] or '_B' in data.columns[3]:
    data = data[['Frame','Time','SMPTE']+data.filter(like=f'_{side[1]}').columns.tolist()]
    data.columns = [col.replace(f'_{side[1]}','') for col in data.columns]
elif '61A' in data.columns[3] or '61B' in data.columns[3]:
    data = data[['Frame','Time','SMPTE']+data.filter(like=f'61{side[1]}').columns.tolist()]
    data.columns = [col.replace(f'61{side[1]}_','') for col in data.columns]
if '_01' in data.columns[3]:
    data.columns = [column for column in data.columns[:3]]+[column[:-5]+column[-2:] for column in data.columns[3:]]
if 'HeadFront' in data.columns[3] or 'RKneeIn' in data.columns[3]:
    mappings = pd.read_csv("data/raw/Sport42MarkersTo64.csv")
    mappings = mappings.set_index(mappings.columns[0])[mappings.columns[1]].to_dict()
    data.columns = [mappings[col[:-2]]+col[-2:] if col[:-2] in mappings.keys() else col for col in data.columns]
    

In [109]:
list(data.columns)[:3]+list(data.columns)[3::3]

['Frame',
 'Time',
 'SMPTE',
 'RKNI X',
 'RKNE X',
 'LKNI X',
 'ARIEL X',
 'LFHD X',
 'RFHD X',
 'STRN X',
 'LFWT X',
 'RFWT X',
 'LSHO X',
 'RSHO X',
 'C7 X',
 'T10 X',
 'LBWT X',
 'RBWT X',
 'BWT X',
 'LHEL X',
 'RHEL X',
 'LANK X',
 'RANK X',
 'LTOE X',
 'RTOE X',
 'LMT5 X',
 'RMT5 X',
 'LPLM X',
 'RPLM X',
 'LIWR X',
 'RIWR X',
 'LOWR X',
 'ROWR X',
 'LIEL X',
 'RIEL X',
 'LELB X',
 'RELB X',
 'LKNE X']

## Working on NaNs

In [110]:
print("Total NaNs")
nans = data[data.columns[(data.isna().sum(axis=0) > 0 ).to_list()]].isna().sum(axis=0).sort_values(ascending=False)
color = '\x1b[32m' if len(nans) == 0 else '\x1b[31m'
print(color+str(nans)+'\x1b[0m '+ ('\nNo Nans' if len(nans)==0 else '\nNans found!'))

Total NaNs
Series([], dtype: float64) 
No Nans


In [111]:
from itertools import groupby
max_nans_per_column = data.apply(lambda col: max(len(list(group))*value for value, group in groupby(col.isna().tolist())))
print("Max contiguous NaNs")
max_nans_per_column = max_nans_per_column[max_nans_per_column > 0].sort_values(ascending=False)
color = '\x1b[32m' if len(max_nans_per_column) == 0 else '\x1b[31m'
print(color+str(max_nans_per_column)+'\x1b[0m ' + ('\nNo Nans' if len(nans)==0 else '\nNans found!'))

Max contiguous NaNs
Series([], dtype: int64) 
No Nans


## Working on Zeros

In [112]:
print('Num Zeros')
zeros = data[data.columns[(data.eq(0.0).sum(axis=0) > 0).to_list()]].eq(0.0).sum(axis=0).sort_values(ascending=False)
color = '\x1b[32m' if len(zeros) == 0 else '\x1b[31m'
print(color+str(zeros)+'\x1b[0m ' + ('\nNo Zeros' if len(zeros)==0 else '\nZeros found!'))
#list(zeros.index)

Num Zeros
Series([], dtype: float64) 
No Zeros


In [113]:
import numpy as np
#for name_col,num_zeros in zeros.items():
#    if num_zeros < 20:
#        data[name_col] = data[name_col].replace(0.0,np.nan).interpolate(method='spline', order=3)

# THIS IS A FIX FOR MISSING QTM AT DATA 58-59
for name_col,num_zeros in zeros.items():
    if num_zeros < 20 and name_col != 'Time' and False:
        data[name_col] = data[name_col].replace(0.0,np.nan).interpolate(method='spline', order=3)

In [114]:
print('Num Zeros')
zeros = data[data.columns[(data.eq(0.0).sum(axis=0) > 0).to_list()]].eq(0.0).sum(axis=0).sort_values(ascending=False)
color = '\x1b[32m' if len(zeros) == 0 else '\x1b[31m'
print(color+str(zeros)+'\x1b[0m ' + ('\nNo Zeros' if len(zeros)==0 else '\nZeros found!'))
#list(zeros.index)

Num Zeros
Series([], dtype: float64) 
No Zeros


In [115]:
from itertools import groupby
max_zeros_per_column = data.apply(lambda col: max(len(list(group))*value for value, group in groupby(col.eq(0.0).tolist())))
print("Max contiguous Zeros")
max_zeros_per_column = max_zeros_per_column[max_zeros_per_column > 0].sort_values(ascending=False)
color = '\x1b[32m' if len(max_zeros_per_column) == 0 else '\x1b[31m'
print(color+str(max_zeros_per_column)+'\x1b[0m ' + ('\nNo Zeros' if len(max_zeros_per_column)==0 else '\nZeros found!'))

Max contiguous Zeros
Series([], dtype: int64) 
No Zeros


## Converting to reduced marker set

![fullbodymarkers](../resources/fullmarkerset.png) ![markerstable](../resources/markersetable.png)

nono smarmella e apri tutto

In [116]:
def map_reduce_num_markers(reducedMarkerNames:list):
    markersMap = {reducedMarkerNames[0]:   ['RHEL','RMT1','RMT5','RTOE'],
                  reducedMarkerNames[1]:   ['LHEL','LMT1','LMT5','LTOE'],
                  reducedMarkerNames[2]:   ['RANK'],
                  reducedMarkerNames[3]:   ['LANK'],
                  reducedMarkerNames[4]:   ['RKNE','RKNI'],
                  reducedMarkerNames[5]:   ['LKNE','LKNI'],
                  reducedMarkerNames[6]:   ['RFWT','RBWT'],
                  reducedMarkerNames[7]:   ['RFWT','LFWT','RBWT','LBWT'],
                  reducedMarkerNames[8]:   ['LFWT','LBWT'],
                  reducedMarkerNames[9]:   ['C6','T5','T10','BWT','STRN','CLAV','FWT'],
                  reducedMarkerNames[10]:  ['RPLM','RTHMB','RINDX','RMID','RPNKY'],
                  reducedMarkerNames[11]:  ['LPLM','LTHMB','LINDX','LMID','LPNKY'],
                  reducedMarkerNames[12]:  ['ROWR','RIWR'],
                  reducedMarkerNames[13]:  ['LOWR','LIWR'],
                  reducedMarkerNames[14]:  ['RELB','RIEL','RFRM'],
                  reducedMarkerNames[15]:  ['LELB','LIEL','LFRM'],
                  reducedMarkerNames[16]:  ['RSHO'],
                  reducedMarkerNames[17]:  ['RSHO','LSHO'],
                  reducedMarkerNames[18]:  ['LSHO'],
                  reducedMarkerNames[19]:  ['ARIEL','RFHD','LFHD','RBHD','LBHD']
                  }
    #   removing nan joints
    for nan_joint_name,_ in max_nans_per_column[::3].items():
        nan_joint_name = nan_joint_name[:-2]
        for markerFullList in markersMap.values():
            if nan_joint_name in markerFullList:
                markerFullList.remove(nan_joint_name)
                break
    # removing many contiguous zeros joints
    for zero_joint_name,contiguous_zeros in max_zeros_per_column[::3].items():
        if contiguous_zeros > 20:
            zero_joint_name = zero_joint_name[:-2]
            for markerFullList in markersMap.values():
                if zero_joint_name in markerFullList:
                    markerFullList.remove(zero_joint_name)
                    break
    originalDataColumns = [colname[:-2] for colname in list(data.iloc[:1,3::3].columns)]
    
    #   removing markers not present in the data markers
    for markerFullList in markersMap.values():
        for markerName in list(reversed(markerFullList)):
            if not markerName in originalDataColumns:
                markerFullList.remove(markerName)

    #   removing asymmetries across body
    for right_side_marker_indx,left_side_marker_indx in [(0,1),(2,3),(4,5),(6,8),(10,11),(12,13),(14,15)]: # indices of every marker on the left side and his symmetric
        markersLeft = markersMap[reducedMarkerNames[left_side_marker_indx]]
        markersRight = markersMap[reducedMarkerNames[right_side_marker_indx]]
        for elem in markersLeft:
            if not 'R'+elem[1:] in markersRight:
                markersLeft.remove(elem)
                if right_side_marker_indx == 6: 
                    markersMap[reducedMarkerNames[7]].remove(elem)
        for elem in markersRight:
            if not 'L'+elem[1:] in markersLeft:
                markersRight.remove(elem)
                if left_side_marker_indx == 8: 
                    markersMap[reducedMarkerNames[7]].remove(elem)

    for i,markerFullList in enumerate(markersMap.values()):
        if len(markerFullList) == 0:
            print(markersMap)
            raise Exception(f"can't reconstruct '{reducedMarkerNames[i]}' of 20 markers, because every marker of the full set is missing!!")
    return markersMap

In [117]:
reducedMarkerNames = ['left_foot',      # 0
                      'right_foot',     # 1          
                      'left_ank',       # 2      
                      'right_ank',      # 3                          
                      'left_knee',      # 4                          
                      'right_knee',     # 5                              
                      'left_hip',       # 6                          
                      'hip_central',    # 7                              
                      'right_hip',      # 8                          
                      'spine',          # 9                       
                      'left_hand',      # 10                           
                      'right_hand',     # 11                               
                      'left_wrist',     # 12                               
                      'right_wrist',    # 13                               
                      'left_elbow',     # 14                               
                      'right_elbow',    # 15                               
                      'left_shoulder',  # 16                               
                      'shoulder_center',# 17                                   
                      'right_shoulder', # 18                                   
                      'head']           # 19                       
fullMarkerNames = [colname[:-2] for colname in list(data.iloc[:1,3::3].columns)]
reduced20MarkersToFullMarkers = map_reduce_num_markers(reducedMarkerNames)
reduced20MarkersToFullMarkersX = {key+'_X': [elem+' X' for elem in reduced20MarkersToFullMarkers[key]] for key in reduced20MarkersToFullMarkers.keys()}
reduced20MarkersToFullMarkersY = {key+'_Y': [elem+' Y' for elem in reduced20MarkersToFullMarkers[key]] for key in reduced20MarkersToFullMarkers.keys()}
reduced20MarkersToFullMarkersZ = {key+'_Z': [elem+' Z' for elem in reduced20MarkersToFullMarkers[key]] for key in reduced20MarkersToFullMarkers.keys()}

posTable = data.iloc[:,3:]

posTableX = posTable.iloc[:,::3]
posTableY = posTable.iloc[:,1::3]
posTableZ = posTable.iloc[:,2::3]

reducedPosTableX = pd.concat([posTableX[reduced20MarkersToFullMarkersX[colName+'_X']].mean(axis=1) for colName in reducedMarkerNames],axis=1,keys=[name+'_X' for name in reducedMarkerNames])
reducedPosTableY = pd.concat([posTableY[reduced20MarkersToFullMarkersY[colName+'_Y']].mean(axis=1) for colName in reducedMarkerNames],axis=1,keys=[name+'_Y' for name in reducedMarkerNames])
reducedPosTableZ = pd.concat([posTableZ[reduced20MarkersToFullMarkersZ[colName+'_Z']].mean(axis=1) for colName in reducedMarkerNames],axis=1,keys=[name+'_Z' for name in reducedMarkerNames])

#reducedPosTableX = (reducedPosTableX - reducedPosTableX.min(axis=None))/(reducedPosTableX.max(axis=None)-reducedPosTableX.min(axis=None))
#reducedPosTableY = (reducedPosTableY - reducedPosTableY.min(axis=None))/(reducedPosTableY.max(axis=None)-reducedPosTableY.min(axis=None))
#reducedPosTableZ = (reducedPosTableZ - reducedPosTableZ.min(axis=None))/(reducedPosTableZ.max(axis=None)-reducedPosTableZ.min(axis=None))

def xyz_tables_to_xyz_columns(tablesList):
    xTable,yTable,zTable = tablesList
    mergedTable = pd.DataFrame()
    for joint in range(xTable.shape[1]):
        mergedTable = pd.concat([mergedTable,xTable.iloc[:,joint],yTable.iloc[:,joint],zTable.iloc[:,joint]],axis=1)
    return mergedTable

reducedPosTable = xyz_tables_to_xyz_columns([reducedPosTableX,reducedPosTableY,reducedPosTableZ])
reducedPosTable = (reducedPosTable - reducedPosTable.min(axis=None))/(reducedPosTable.max(axis=None)-reducedPosTable.min(axis=None))
reducedPosTable.head()

,left_foot_X,left_foot_Y,left_foot_Z,right_foot_X,right_foot_Y,right_foot_Z,left_ank_X,left_ank_Y,left_ank_Z,right_ank_X,...,left_shoulder_Z,shoulder_center_X,shoulder_center_Y,shoulder_center_Z,right_shoulder_X,right_shoulder_Y,right_shoulder_Z,head_X,head_Y,head_Z
0,0.805175,0.366991,0.002240,0.797031,0.284102,0.001988,0.842428,0.374363,0.026078,0.835740,...,0.459099,0.756410,0.329404,0.459421,0.755606,0.276202,0.459743,0.736310,0.327883,0.542268
1,0.805204,0.367010,0.002261,0.797064,0.284065,0.002030,0.842511,0.374392,0.026045,0.835863,...,0.460879,0.757879,0.329643,0.461207,0.757203,0.276241,0.461535,0.738242,0.328122,0.544294
2,0.805228,0.367035,0.002267,0.797098,0.284022,0.002057,0.842576,0.374430,0.026004,0.835999,...,0.462642,0.759315,0.329885,0.462938,0.758764,0.276305,0.463234,0.740150,0.328365,0.546208
3,0.805247,0.367054,0.002276,0.797141,0.283939,0.002115,0.842624,0.374472,0.025954,0.836143,...,0.464353,0.760707,0.330137,0.464607,0.760328,0.276376,0.464861,0.742021,0.328613,0.548051
4,0.805259,0.367062,0.002293,0.797197,0.283829,0.002177,0.842653,0.374500,0.025944,0.836264,...,0.466023,0.762049,0.330387,0.466218,0.761861,0.276475,0.466413,0.743847,0.328854,0.549798


## Writing to files final clean

In [118]:
import os

PROCESSED_PATH = 'data/reprocessed/'

dest_dir = os.path.join(PROCESSED_PATH,folder)
if not os.path.exists(dest_dir): 
    os.mkdir(dest_dir)
dest_file_path = os.path.join(dest_dir,trial+'_frag'+fragId+'.csv')
if not os.path.isfile(dest_file_path) or True:
    reducedPosTable.to_csv(dest_file_path,index=False)

## Plotting reduced markers

In [52]:
from matplotlib import pyplot as plt
%matplotlib
from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
if sampleAnno < 45:
    posTableY = reducedPosTable.iloc[:,::3]
    posTableZ = reducedPosTable.iloc[:,1::3]
    posTableX = reducedPosTable.iloc[:,2::3]
else:
    posTableX = reducedPosTable.iloc[:,::3]
    posTableY = reducedPosTable.iloc[:,1::3]
    posTableZ = reducedPosTable.iloc[:,2::3]

jointsFrom = [1, 3, 5, 7, 2, 4, 6, 9, 8,10,11,13,15,17,12,14,16,19,18]
jointsTo =   [3, 5, 7, 8, 4, 6, 9, 8,10,18,13,15,17,18,14,16,19,18,20]
jointsFrom = list(map(lambda x: x-1,jointsFrom))
jointsTo = list(map(lambda x: x-1,jointsTo))
edges = np.array(list(zip(jointsFrom,jointsTo))+list(zip(jointsTo,jointsFrom)))
adjacencyMatrix = np.zeros((20,20),dtype=bool)
adjacencyMatrix[edges[:,0],edges[:,1]] = True

plt.close("all")
fig = plt.figure()
ax:plt.Axes = fig.add_subplot(111, projection='3d')

minMax = np.zeros((2,3))
minMax[0,:] = [np.nanmin(posTableX.values),np.nanmin(posTableY.values),np.nanmin(posTableZ.values)]
minMax[1,:] = [np.nanmax(posTableX.values),np.nanmax(posTableY.values),np.nanmax(posTableZ.values)]

# Set the window title
fig.canvas.manager.window.title("3D Movement\t(Scroll with mouse wheel)")

# Set the initial time index
time_index = 0

# Function to update the plot based on the slider value
def update_plot(val):
    ax.cla()  # Clear the previous plot
    
    # Filter the data based on the current time index
    filteredX = posTableX.iloc[val]
    filteredY = posTableY.iloc[val]
    filteredZ = posTableZ.iloc[val]
    
    ax.scatter(filteredX,filteredY,filteredZ)

    # Add edges based on the weight matrix
    for i in range(20):
        for j in range(i + 1, 20):
            if adjacencyMatrix[i,j]:
                ax.plot([filteredX[i], filteredX[j]],
                        [filteredY[i], filteredY[j]],
                        [filteredZ[i], filteredZ[j]],
                        color='gray', linestyle='-', linewidth=1)

    ax.set_xlim([-0.1,0.8])#minMax[0,0],minMax[1,0]])
    ax.set_ylim([-0.1,0.8])#minMax[0,1],minMax[1,1]])
    ax.set_zlim([minMax[0,2],minMax[1,2]])

    ax.set_xlabel('X', fontsize=12)
    ax.set_ylabel('Y', fontsize=12)
    ax.set_zlabel('Z', fontsize=12)
    ax.set_title("Movement "+str(sampleAnno))

    fig.canvas.draw_idle()

# Create a slider widget
slider_ax = plt.axes([0.2, 0.03, 0.6, 0.03])
maxValue = posTable.shape[0]-1
slider = Slider(slider_ax, 'TimeIndex:', 0, maxValue, valinit=time_index, valstep=1)


# Define a function to update the slider value with the mouse wheel
def on_scroll(event):
    if event.button == 'down':
        if slider.val + slider.valstep*2 <= maxValue:
            slider.set_val(slider.val + slider.valstep*2)
    elif event.button == 'up':
        if slider.val - slider.valstep*2 >= 0:
            slider.set_val(slider.val - slider.valstep*2)
        

# Connect the mouse wheel event to the function
fig.canvas.mpl_connect('scroll_event', on_scroll)


# Register the update_plot function with the slider widget
slider.on_changed(update_plot)

# Initial plot
update_plot(time_index)

# Show the plot
plt.show()

Using matplotlib backend: TkAgg


## Plotting full markers

In [51]:
from matplotlib import pyplot as plt
%matplotlib
from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
if sampleAnno < 45:
    posTableY = posTable.iloc[:,::3]
    posTableZ = posTable.iloc[:,1::3]
    posTableX = posTable.iloc[:,2::3]
else:
    posTableX = posTable.iloc[:,::3]
    posTableY = posTable.iloc[:,1::3]
    posTableZ = posTable.iloc[:,2::3]
    

plt.close("all")
fig = plt.figure()
ax:plt.Axes = fig.add_subplot(111, projection='3d')

minMax = np.zeros((2,3))
minMax[0,:] = [np.nanmin(posTableX.values),np.nanmin(posTableY.values),np.nanmin(posTableZ.values)]
minMax[1,:] = [np.nanmax(posTableX.values),np.nanmax(posTableY.values),np.nanmax(posTableZ.values)]

# Set the window title
fig.canvas.manager.window.title("3D Movement\t(Scroll with mouse wheel)")

# Set the initial time index
time_index = 0
column_names = [ col[:-2] for col in list(posTable.columns)[::3] ]

# Function to update the plot based on the slider value
def update_plot(val):
    ax.cla()  # Clear the previous plot
    
    # Filter the data based on the current time index
    filteredX = posTableX.iloc[val]
    filteredY = posTableY.iloc[val]
    filteredZ = posTableZ.iloc[val]
    
    ax.scatter(filteredX,filteredY,filteredZ)

    for i, name in enumerate(column_names):
        ax.text(filteredX[i], filteredY[i], filteredZ[i], name, fontsize=8, ha='left')


    ax.set_xlim([minMax[0,0],minMax[1,0]])
    ax.set_ylim([minMax[0,1],minMax[1,1]])
    ax.set_zlim([minMax[0,2],minMax[1,2]])

    ax.set_xlabel('X', fontsize=12)
    ax.set_ylabel('Y', fontsize=12)
    ax.set_zlabel('Z', fontsize=12)
    #ax.set_title("Movement "+str(picked))

    fig.canvas.draw_idle()

# Create a slider widget
slider_ax = plt.axes([0.2, 0.03, 0.6, 0.03])
maxValue = posTable.shape[0]-1
slider = Slider(slider_ax, 'TimeIndex:', 0, maxValue, valinit=time_index, valstep=1)


# Define a function to update the slider value with the mouse wheel
def on_scroll(event):
    if event.button == 'down':
        if slider.val + slider.valstep*2 <= maxValue:
            slider.set_val(slider.val + slider.valstep*2)
    elif event.button == 'up':
        if slider.val - slider.valstep*2 >= 0:
            slider.set_val(slider.val - slider.valstep*2)
        

# Connect the mouse wheel event to the function
fig.canvas.mpl_connect('scroll_event', on_scroll)


# Register the update_plot function with the slider widget
slider.on_changed(update_plot)

# Initial plot
update_plot(time_index)

# Show the plot
plt.show()


Using matplotlib backend: TkAgg


In [19]:
from matplotlib import pyplot as plt
%matplotlib
from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d import Axes3D
import numpy as np


plt.close("all")
fig = plt.figure()
ax:plt.Axes = fig.add_subplot(111, projection='3d')

minMax = np.zeros((2,3))
minMax[0,:] = [np.nanmin(posTableX.values),np.nanmin(posTableY.values),np.nanmin(posTableZ.values)]
minMax[1,:] = [np.nanmax(posTableX.values),np.nanmax(posTableY.values),np.nanmax(posTableZ.values)]

minMax[0,:] = [np.nanmin(posTableX.iloc[:1,:].values),np.nanmin(posTableY.iloc[:1,:].values),np.nanmin(posTableZ.iloc[:1,:].values)]
minMax[1,:] = [np.nanmax(posTableX.iloc[:1,:].values),np.nanmax(posTableY.iloc[:1,:].values),np.nanmax(posTableZ.iloc[:1,:].values)]

# Set the window title
fig.canvas.manager.window.title("3D Movement\t(Scroll with mouse wheel)")

# Set the initial time index
time_index = 0

# Function to update the plot based on the slider value
def update_plot(val):
    ax.cla()  # Clear the previous plot
    
    # Filter the data based on the current time index
    filteredX = posTableX.iloc[val]
    filteredY = posTableY.iloc[val]
    filteredZ = posTableZ.iloc[val]
    
    ax.scatter(filteredX,filteredY,filteredZ,s=0)
    
    for i, col_name in enumerate(posTableX.columns):
        ax.text(filteredX[i], filteredY[i], filteredZ[i], col_name[:-2], horizontalalignment='center',fontsize=6, color='black')

    ax.set_xticks([minMax[0,0],minMax[1,0]])
    ax.set_yticks([minMax[0,1],minMax[1,1]])
    ax.set_zticks([minMax[0,2],minMax[1,2]])
    ax.set_xlim([minMax[0,0],minMax[1,0]])
    ax.set_ylim([minMax[0,1],minMax[1,1]])
    ax.set_zlim([minMax[0,2],minMax[1,2]])
    
    fig.canvas.draw_idle()

# Create a slider widget
slider_ax = plt.axes([0.2, 0.03, 0.6, 0.03])
maxValue = posTable.shape[0]-1
slider = Slider(slider_ax, 'TimeIndex:', 0, maxValue, valinit=time_index, valstep=1)


# Define a function to update the slider value with the mouse wheel
def on_scroll(event):
    if event.button == 'down':
        if slider.val + slider.valstep*2 <= maxValue:
            slider.set_val(slider.val + slider.valstep*2)
    elif event.button == 'up':
        if slider.val - slider.valstep*2 >= 0:
            slider.set_val(slider.val - slider.valstep*2)
        

# Connect the mouse wheel event to the function
fig.canvas.mpl_connect('scroll_event', on_scroll)


# Register the update_plot function with the slider widget
slider.on_changed(update_plot)

# Initial plot
update_plot(time_index)

# Show the plot
plt.show()


Using matplotlib backend: TkAgg


In [242]:
plt.close('all')

# All in One

In [195]:
from matplotlib import pyplot as plt
%matplotlib
from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

def visualize_a_sample(posTable, sample):
    posTableX = posTable.iloc[:,::3]
    posTableY = posTable.iloc[:,2::3]
    posTableZ = posTable.iloc[:,1::3]
    jointsFrom = [1, 3, 5, 7, 2, 4, 6, 9, 8,10,11,13,15,17,12,14,16,19,18]
    jointsTo =   [3, 5, 7, 8, 4, 6, 9, 8,10,18,13,15,17,18,14,16,19,18,20]
    jointsFrom = list(map(lambda x: x-1,jointsFrom))
    jointsTo = list(map(lambda x: x-1,jointsTo))
    edges = np.array(list(zip(jointsFrom,jointsTo))+list(zip(jointsTo,jointsFrom)))
    adjacencyMatrix = np.zeros((20,20),dtype=bool)
    adjacencyMatrix[edges[:,0],edges[:,1]] = True

    plt.close('all')
    fig = plt.figure(figsize=(10,8))
    ax:plt.Axes = fig.add_subplot(111, projection='3d')
    ax.view_init(elev=0, azim=0)
    fig.canvas.manager.full_screen_toggle()

    
    minMax = np.zeros((2,3))
    minMax[0,:] = [posTableX.values.min(),posTableY.values.min(),posTableZ.values.min()]
    minMax[1,:] = [posTableX.values.max(),posTableY.values.max(),posTableZ.values.max()]

    # Set appropriate axis limits
    ax.set_xlim([minMax[0,0],minMax[1,0]])
    ax.set_ylim([0,1])
    ax.set_zlim([minMax[0,2],minMax[1,2]])

    # Set the window title
    fig.canvas.manager.window.title("3D Movement\t(Scroll with mouse wheel)")

    # Set the initial time index
    time_index = 0
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'yellow', 'cyan']

    # Function to update the plot based on the slider value
    def update_plot(val):
        ax.cla()  # Clear the previous plot

        # Filter the data based on the current time index
        filteredX = posTableX.iloc[val]
        filteredY = posTableY.iloc[val]
        filteredZ = posTableZ.iloc[val]

        ax.scatter(filteredX,filteredY,filteredZ)

        # Add edges based on the weight matrix
        for i in range(20):
            for j in range(i + 1, 20):
                if adjacencyMatrix[i,j]:
                    ax.plot([filteredX[i], filteredX[j]],
                            [filteredY[i], filteredY[j]],
                            [filteredZ[i], filteredZ[j]],
                            color='gray', linestyle='-', linewidth=1)


        ax.set_xlim([minMax[0,0],minMax[1,0]])
        ax.set_ylim([0,1])
        ax.set_zlim([minMax[0,2],minMax[1,2]])

        ax.set_xlabel('X', fontsize=12)
        ax.set_ylabel('Y', fontsize=12)
        ax.set_zlabel('Z', fontsize=12)
        ax.set_title("Movement "+str(sample))

        fig.canvas.draw_idle()

    # Create a slider widget
    slider_ax = plt.axes([0.2, 0.03, 0.6, 0.03])
    maxValue = posTableX.shape[0]-1
    slider = Slider(slider_ax, 'TimeIndex:', 0, maxValue, valinit=time_index, valstep=1)


    # Define a function to update the slider value with the mouse wheel
    def on_scroll(event):
        if event.button == 'down':
            if slider.val + slider.valstep <= maxValue:
                slider.set_val(slider.val + slider.valstep)
        elif event.button == 'up':
            if slider.val - slider.valstep >= 0:
                slider.set_val(slider.val - slider.valstep)


    # Connect the mouse wheel event to the function
    fig.canvas.mpl_connect('scroll_event', on_scroll)


    # Register the update_plot function with the slider widget
    slider.on_changed(update_plot)

    # Initial plot
    update_plot(time_index)

    # Show the plot
    plt.show(block=True)

# Function to find the maximum consecutive zeros values from a mask series
def max_consecutive_zeros(series):
        consecutive_true_count = 0
        max_consecutive_true_count = 0

        for value in series:
            if value:
                consecutive_true_count += 1
                if consecutive_true_count > max_consecutive_true_count:
                    max_consecutive_true_count = consecutive_true_count
            else:
                consecutive_true_count = 0

        return max_consecutive_true_count


Using matplotlib backend: TkAgg


In [206]:
import csv
import os
import pandas as pd
from tqdm.notebook import tqdm
from itertools import groupby

def custom_header_reader(file:'TextIOWrapper'):
    csvReader = csv.reader(file,delimiter='\t')
    for row, text in enumerate(csvReader):
        if row == 2: numMarkers = int(text[-1])
        elif row == 10: 
            columnNames = text[:-1]
            if columnNames[0] == 'Frame': 
                break
        elif row == 11: columnNames = text[:-1]; break
    return numMarkers, columnNames, csvReader

def line_reader(csvReader,fromSecond,toSecond):
    for line in csvReader:
        if fromSecond <= float(line[1]) <= toSecond:
            yield line[:len(columnNames)]
        elif float(line[1]) > toSecond:
            break


ANNOTATIONS_PATH = 'data'
RAW_PATH = ANNOTATIONS_PATH+'/raw'
#with open(ANNOTATIONS_PATH+'/annotations.txt','r') as file:
#    annotations = file.read().splitlines()
with open(ANNOTATIONS_PATH+'/annotationsVSingle.txt','r') as file:
    annotations = file.read().splitlines()[1:]#[line for line in file.read().splitlines() if 'no val' in line]
    #print(len(annotations))
    
for sampleAnno in tqdm(range(len(annotations)),desc="Compressing markers and saving to csv reprocessed"):
    if '(A)' in annotations[sampleAnno] or '(B)' in annotations[sampleAnno]:
        folder, trial, fragId, side, OoM, startSec, endSec = annotations[sampleAnno].replace(' ','').split(',')[:7]
    else:
        folder, trial, fragId, OoM, startSec, endSec = annotations[sampleAnno].replace(' ','').split(',')[:6]
    sampleData = os.path.join(RAW_PATH,folder,trial+'.tsv')
    with open(sampleData,'r') as file:
        numMarkers, columnNames, readerBuffer = custom_header_reader(file)
        data = pd.DataFrame(line_reader(readerBuffer,float(startSec),float(endSec)),columns=columnNames).astype(dict(zip(columnNames,[int,float,str]+[float]*len(columnNames[3:]))))

    if data['Time'].iloc[0] > float(startSec) or data['Time'].iloc[-1] < float(endSec):
        raise Exception(f'\x1b[31mNOT FULL FRAMES!!! data starts at: {data["Time"].iloc[0]} of {float(startSec)} and ends at {data["Time"].iloc[-1]} of {float(endSec)}\x1b[0m')
    else:
        pass


    if '_A' in data.columns[3] or '_B' in data.columns[3]:
        data = data[['Frame','Time','SMPTE']+data.filter(like=f'_{side[1]}').columns.tolist()]
        data.columns = [col.replace(f'_{side[1]}','') for col in data.columns]
    elif '61A' in data.columns[3] or '61B' in data.columns[3]:
        data = data[['Frame','Time','SMPTE']+data.filter(like=f'61{side[1]}').columns.tolist()]
        data.columns = [col.replace(f'61{side[1]}_','') for col in data.columns]
    if '_01' in data.columns[3]:
        data.columns = [column for column in data.columns[:3]]+[column[:-5]+column[-2:] for column in data.columns[3:]]
    if 'HeadFront' in data.columns[3] or 'RKneeIn' in data.columns[3]:
        mappings = pd.read_csv("data/raw/Sport42MarkersTo64.csv")
        mappings = mappings.set_index(mappings.columns[0])[mappings.columns[1]].to_dict()
        data.columns = [mappings[col[:-2]]+col[-2:] if col[:-2] in mappings.keys() else col for col in data.columns]

    nans = data[data.columns[(data.isna().sum(axis=0) > 0 ).to_list()]].isna().sum(axis=0).sort_values(ascending=False)



    max_nans_per_column = data.apply(lambda col: max(len(list(group))*value for value, group in groupby(col.isna().tolist())))

    max_nans_per_column = max_nans_per_column[max_nans_per_column > 0].sort_values(ascending=False)


    zeros = data[data.columns[(data.eq(0.0).sum(axis=0) > 0).to_list()]].eq(0.0).sum(axis=0).sort_values(ascending=False)

    # THIS IS A FIX FOR MISSING QTM AT DATA 58-59
    if len(zeros.index) > 15:
        for name_col,num_zeros in zeros.items():
            if num_zeros < 50:
                data[name_col] = data[name_col].replace(0.0,np.nan).interpolate(method='spline', order=3)
    if sampleAnno == 1:
        for name_col,num_zeros in zeros.items():
            if num_zeros < 10:
                data[name_col] = data[name_col].replace(0.0,np.nan).interpolate(method='spline', order=3)
    #for name_col,num_zeros in zeros.items():
    #    if num_zeros < 20:
    #        data[name_col] = data[name_col].replace(0.0,np.nan).interpolate(method='spline', order=3)

    zeros = data[data.columns[(data.eq(0.0).sum(axis=0) > 0).to_list()]].eq(0.0).sum(axis=0).sort_values(ascending=False)

    max_zeros_per_column = data.apply(lambda col: max(len(list(group))*value for value, group in groupby(col.eq(0.0).tolist())))
    max_zeros_per_column = max_zeros_per_column[max_zeros_per_column > 0].sort_values(ascending=False)

    def map_reduce_num_markers(reducedMarkerNames:list):
        markersMap = {reducedMarkerNames[0]:   ['RHEL','RMT1','RMT5','RTOE'],
                      reducedMarkerNames[1]:   ['LHEL','LMT1','LMT5','LTOE'],
                      reducedMarkerNames[2]:   ['RANK'],
                      reducedMarkerNames[3]:   ['LANK'],
                      reducedMarkerNames[4]:   ['RKNE','RKNI'],#,'RSHN','RShinFrontHigh'],
                      reducedMarkerNames[5]:   ['LKNE','LKNI'],#,'LSHN','LShinFrontHigh'],
                      reducedMarkerNames[6]:   ['RFWT','RBWT'],
                      reducedMarkerNames[7]:   ['RFWT','LFWT','RBWT','LBWT'],
                      reducedMarkerNames[8]:   ['LFWT','LBWT'],
                      reducedMarkerNames[9]:   ['C6','C7','T5','T10','BWT','STRN','CLAV','FWT'],
                      reducedMarkerNames[10]:  ['RPLM','RTHMB','RINDX','RMID','RPNKY'],
                      reducedMarkerNames[11]:  ['LPLM','LTHMB','LINDX','LMID','LPNKY'],
                      reducedMarkerNames[12]:  ['ROWR','RIWR'],
                      reducedMarkerNames[13]:  ['LOWR','LIWR'],
                      reducedMarkerNames[14]:  ['RELB','RIEL','RFRM'],
                      reducedMarkerNames[15]:  ['LELB','LIEL','LFRM'],
                      reducedMarkerNames[16]:  ['RSHO'],
                      reducedMarkerNames[17]:  ['RSHO','LSHO'],#'C7'],
                      reducedMarkerNames[18]:  ['LSHO'],
                      reducedMarkerNames[19]:  ['ARIEL','RFHD','LFHD','RBHD','LBHD']
                      }
        
        # removing nan joints
        for nan_joint_name,_ in max_nans_per_column[::3].items():
            nan_joint_name = nan_joint_name[:-2]
            for markerFullList in markersMap.values():
                if nan_joint_name in markerFullList:
                    markerFullList.remove(nan_joint_name)
                    break

        # removing many contiguous zeros joints
        for zero_joint_name,contiguous_zeros in max_zeros_per_column[::3].items():
            if contiguous_zeros > 20:
                zero_joint_name = zero_joint_name[:-2]
                for markerFullList in markersMap.values():
                    if zero_joint_name in markerFullList:
                        markerFullList.remove(zero_joint_name)
                        break
        originalDataColumns = [colname[:-2] for colname in list(data.iloc[:1,3::3].columns)]

        # removing markers not present in the data markers
        for markerFullList in markersMap.values():
            for markerName in list(reversed(markerFullList)):
                if not markerName in originalDataColumns:
                    markerFullList.remove(markerName)

        #   removing asymmetries across body
        for right_side_marker_indx,left_side_marker_indx in [(0,1),(2,3),(4,5),(6,8),(10,11),(12,13),(14,15)]: # indices of every marker on the left side and his symmetric
            markersLeft = markersMap[reducedMarkerNames[left_side_marker_indx]]
            markersRight = markersMap[reducedMarkerNames[right_side_marker_indx]]
            for elem in markersLeft:
                if not 'R'+elem[1:] in markersRight:
                    markersLeft.remove(elem)
                    if right_side_marker_indx == 6: 
                        markersMap[reducedMarkerNames[7]].remove(elem)
            for elem in markersRight:
                if not 'L'+elem[1:] in markersLeft:
                    markersRight.remove(elem)
                    if left_side_marker_indx == 8: 
                        markersMap[reducedMarkerNames[7]].remove(elem)

        for i,markerFullList in enumerate(markersMap.values()):
            if len(markerFullList) == 0:
                print(markersMap)
                raise Exception(f"can't reconstruct '{reducedMarkerNames[i]}' of 20 markers, because every marker of the full set is missing!!")
        return markersMap

    reducedMarkerNames = ['right_foot',      # 1
                          'left_foot',       # 2        
                          'right_ank',       # 3      
                          'left_ank',        # 4                          
                          'right_knee',      # 5                          
                          'left_knee',       # 6                              
                          'right_hip',       # 7                          
                          'hip_central',     # 8                              
                          'left_hip',        # 9                          
                          'spine',           # 10                       
                          'right_hand',      # 11                           
                          'left_hand',       # 12                               
                          'right_wrist',     # 13                               
                          'left_wrist',      # 14                               
                          'right_elbow',     # 15                               
                          'left_elbow',      # 16                               
                          'right_shoulder',  # 17                               
                          'shoulder_center', # 18                                   
                          'left_shoulder',   # 19                                   
                          'head']            # 20                       
    fullMarkerNames = [colname[:-2] for colname in list(data.iloc[:1,3::3].columns)]
    reduced20MarkersToFullMarkers = map_reduce_num_markers(reducedMarkerNames)
    reduced20MarkersToFullMarkersX = {key+'_X': [elem+' X' for elem in reduced20MarkersToFullMarkers[key]] for key in reduced20MarkersToFullMarkers.keys()}
    reduced20MarkersToFullMarkersY = {key+'_Y': [elem+' Y' for elem in reduced20MarkersToFullMarkers[key]] for key in reduced20MarkersToFullMarkers.keys()}
    reduced20MarkersToFullMarkersZ = {key+'_Z': [elem+' Z' for elem in reduced20MarkersToFullMarkers[key]] for key in reduced20MarkersToFullMarkers.keys()}

    posTable = data.iloc[:,3:]

    posTableX = posTable.iloc[:,::3]
    posTableY = posTable.iloc[:,1::3]
    posTableZ = posTable.iloc[:,2::3]

    reducedPosTableX = pd.concat([posTableX[reduced20MarkersToFullMarkersX[colName+'_X']].mean(axis=1) for colName in reducedMarkerNames],axis=1,keys=[name+'_X' for name in reducedMarkerNames])
    reducedPosTableY = pd.concat([posTableY[reduced20MarkersToFullMarkersY[colName+'_Y']].mean(axis=1) for colName in reducedMarkerNames],axis=1,keys=[name+'_Y' for name in reducedMarkerNames])
    reducedPosTableZ = pd.concat([posTableZ[reduced20MarkersToFullMarkersZ[colName+'_Z']].mean(axis=1) for colName in reducedMarkerNames],axis=1,keys=[name+'_Z' for name in reducedMarkerNames])

    def xyz_tables_to_xyz_columns(tablesList):
        xTable,yTable,zTable = tablesList
        mergedTable = pd.DataFrame()
        for joint in range(xTable.shape[1]):
            mergedTable = pd.concat([mergedTable,xTable.iloc[:,joint],yTable.iloc[:,joint],zTable.iloc[:,joint]],axis=1)
        return mergedTable

    reducedPosTable = xyz_tables_to_xyz_columns([reducedPosTableX,reducedPosTableY,reducedPosTableZ])
    reducedPosTable = (reducedPosTable - reducedPosTable.min(axis=None))/(reducedPosTable.max(axis=None)-reducedPosTable.min(axis=None))
    #reducedPosTable = reducedPosTable.rolling(window=2).mean().fillna(method='bfill')
    
    if reducedPosTable.eq(0.0).apply(max_consecutive_zeros).max() > 1:
        print(sampleAnno, reducedPosTable.eq(0.0).apply(max_consecutive_zeros).max())
        visualize_a_sample(reducedPosTable,sampleAnno)

    PROCESSED_PATH = 'data/reprocessed/'

    dest_dir = os.path.join(PROCESSED_PATH,folder)
    if not os.path.exists(dest_dir): 
        os.mkdir(dest_dir)
    dest_file_path = os.path.join(dest_dir,trial+'_frag'+fragId+'.csv')
    if not os.path.isfile(dest_file_path) or True:
        reducedPosTable.to_csv(dest_file_path,index=False)

Compressing markers and saving to csv reprocessed:   0%|          | 0/60 [00:00<?, ?it/s]